In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd 
import numpy as np

In [5]:
datafile_train="Hazard_train.csv"
datafile_test="Hazard_test_share.csv"
bd_train=pd.read_csv(datafile_train)
bd_test=pd.read_csv(datafile_test)

In [6]:
bd_train.head()

,Id,Hazard,T1_V1,T1_V2,T1_V3,T1_V4,T1_V5,T1_V6,T1_V7,T1_V8,...,T2_V6,T2_V7,T2_V8,T2_V9,T2_V10,T2_V11,T2_V12,T2_V13,T2_V14,T2_V15
0,83642,2,10,23,4,N,K,Y,B,D,...,2,37,1,11,2,Y,N,E,2,1
1,92530,6,3,18,2,B,H,N,B,B,...,2,34,1,12,7,Y,N,C,3,3
2,86131,4,5,20,4,B,K,N,B,B,...,1,40,1,17,4,Y,N,D,1,1
3,88666,1,4,7,5,N,H,N,B,B,...,2,37,1,11,6,Y,N,D,2,1
4,11122,21,10,21,6,N,C,Y,B,C,...,2,40,1,6,6,N,N,D,5,5


In [7]:
cat_cols=bd_train.select_dtypes(['object']).columns

In [8]:
cat_cols

Index(['T1_V4', 'T1_V5', 'T1_V6', 'T1_V7', 'T1_V8', 'T1_V9', 'T1_V11',
       'T1_V12', 'T1_V15', 'T1_V16', 'T1_V17', 'T2_V3', 'T2_V5', 'T2_V11',
       'T2_V12', 'T2_V13'],
      dtype='object')

In [9]:
bd_train['data']='train'
bd_test['data']='test'
all_data=pd.concat([bd_train,bd_test],axis=0,sort=False)

In [10]:
all_data.shape

(50999, 35)

In [11]:
for col in cat_cols:
    k=all_data[col].value_counts()
    cats=k[k>=100].index[:-1]
    
    for cat in cats:
        name=col+'_'+cat
        all_data[name]=(all_data[col]==cat).astype(int)
        
    del all_data[col]

In [12]:
all_data.shape

(50999, 94)

In [13]:
all_data.head()

,Id,Hazard,T1_V1,T1_V2,T1_V3,T1_V10,T1_V13,T1_V14,T2_V1,T2_V2,...,T2_V5_A,T2_V5_B,T2_V5_C,T2_V5_D,T2_V11_Y,T2_V12_N,T2_V13_E,T2_V13_A,T2_V13_C,T2_V13_D
0,83642,2.0,10,23,4,2,10,0,52,14,...,0,0,1,0,1,1,1,0,0,0
1,92530,6.0,3,18,2,8,10,3,39,14,...,1,0,0,0,1,1,0,0,1,0
2,86131,4.0,5,20,4,7,15,1,66,10,...,0,0,1,0,1,1,0,0,0,1
3,88666,1.0,4,7,5,8,15,3,48,11,...,0,1,0,0,1,1,0,0,0,1
4,11122,21.0,10,21,6,12,10,1,42,11,...,1,0,0,0,0,1,0,0,0,1


In [14]:
x_train=all_data.drop(['Id','Hazard','data'],axis=1)[all_data['data']=='train']
y_train=all_data['Hazard'][all_data['data']=='train']

In [15]:
x_train.shape, y_train.shape

((40799, 91), (40799,))

In [16]:
x_test=all_data.drop(['Id','Hazard','data'],axis=1)[all_data['data']=='test']

In [17]:
import xgboost as xgb



In [18]:
dtrain = xgb.DMatrix(x_train, label=y_train)

In [19]:
params = {
    'objective': 'count:poisson',
    'eval_metric': 'poisson-nloglik'
}

In [20]:
model = xgb.train(params, dtrain)

In [21]:
dtest = xgb.DMatrix(x_test)

In [22]:
y_pred = model.predict(dtest)


In [27]:
submissions = model.predict(dtest)


In [28]:
submissions.to_csv('submission.csv',index=False)

AttributeError: 'numpy.ndarray' object has no attribute 'to_csv'